### creating chat memory 

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from openai import OpenAI
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
import json
from langchain.docstore.document import Document
from langchain.prompts.chat import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import re
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI
import os 
from dotenv import load_dotenv
load_dotenv(".env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# -------------------------------------
# Step 1: Q&A Data from JSON
# -------------------------------------
with open("merged_data.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# Combine each Q&A pair into a single document
documents = []
for qa in qa_data:
    content = f"Question: {qa[0]}\nAnswer: {qa[1]}"
    documents.append(Document(page_content=content))

# 1. Configure a splitter that only subdivides if a chunk exceeds 1000 chars.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # target <=1 000 chars per chunk
    chunk_overlap=200,      # keep 200 chars overlap for context
    length_function=len,    # measure by character count
)

# 2. Apply it—short docs (under 1000 chars) remain intact; long ones get split.
chunked_docs = text_splitter.split_documents(documents)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(documents=chunked_docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,
    openai_api_key=OPENAI_API_KEY,
)

/var/folders/z5/_t6p92zx3t528nckxnsfts4c0000gn/T/ipykernel_33777/218750787.py:59: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [12]:
from langchain.memory import ConversationBufferMemory
from openai import OpenAI
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import json
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import create_history_aware_retriever
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage
from langchain_core.prompts import MessagesPlaceholder
import re

def print_answer(answer):
    # Split on end‑of‑sentence punctuation (keeping the punctuation)
    sentences = re.split(r'(?<=[\?\.\!])\s+', answer.strip())

    # Print each sentence on its own line
    for s in sentences:
        print(s)
def gen_llm():
    contextualize_q_system_prompt = """You are an assistant for question-answering tasks. \
    Use the following pieces of retrieved context to answer the question. \
    If you don't know the answer, just say that you don't know. \
    Use three sentences maximum and keep the answer concise.\{context}"""

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    qa_system_prompt = """
        You are a senior technical interviewer.

        Ask from user an interview question, based on the Relevant Knowledge‑Base Excerpts and given input:
        {input}
        if you don't find a question related to the input, just say that you don't know.
        {context}
        Question:
        """
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain


In [ ]:
def evaluate_llm():
    contextualize_q_system_prompt = """You are an assistant for question-answering tasks. \
    Use the following pieces of retrieved context to answer the question. \
    If you don't know the answer, just say that you don't know. \
    Use three sentences maximum and keep the answer concise.{context}'"""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")])
    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
    qa_system_prompt = """
        You are a senior technical interviewer assistant.

        Interview Question:
        {input}

        Candidate’s Answer:
        {user_answer}

        Relevant Knowledge‑Base Excerpts:
        {context}
        Evaluate the candidate’s answer based on the following criteria:
            a) Score the candidate’s answer from 1 (poor) to 10 (excellent).  
            b) Provide concise, actionable feedback on how to improve.  
            c) Supply the “model” (ideal) answer.  
        Format your response as:
            Score: <number>  
            Feedback: <text>  
            Model Answer: <text>

        Begin now.
        """
    qa_prompt = ChatPromptTemplate.from_messages(
        [("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("system", "{input}"),
        ("human", "Candidate’s Answer: {user_answer}")])
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain


In [14]:
from langchain.memory import ConversationBufferMemory
from openai import OpenAI
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_openai import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import json
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import create_history_aware_retriever
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage
from langchain_core.prompts import MessagesPlaceholder
import re

def print_answer(answer):
    # Split on end‑of‑sentence punctuation (keeping the punctuation)
    sentences = re.split(r'(?<=[\?\.\!])\s+', answer.strip())

    # Print each sentence on its own line
    for s in sentences:
        print(s)
def comunicate_llm():
    contextualize_q_system_prompt = """You are an assistant for question-answering tasks. \
    Use the following pieces of retrieved context to answer the question. \
    If you don't know the answer, just say that you don't know. \
    Use three sentences maximum and keep the answer concise.\{context}"""

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )

    qa_system_prompt = """
        You are a senior technical interviewer.
        communicate with the user, based on the Relevant Knowledge‑Base Excerpts and given input and return Answer to the users:
        {input}
        if you don't find a question related to the input, just say that you don't know.
        {context}
        Answer:
        """
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain


In [ ]:
from langchain_core.messages import HumanMessage

chat_history = []
question = "Generate new question about data science"
generated_question = gen_llm().invoke({"input":question,"chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), generated_question["answer"]])
question = generated_question['answer']

In [ ]:
print_answer(question)

What are some common techniques for handling missing data in a dataset?


In [17]:
answer = """Ptorch, scikit learn, numpy and pandas Library."""
evaluate = evaluate_llm().invoke({"input": question, "context": "", "user_answer": answer, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=answer), evaluate["answer"]])
print_answer(evaluate["answer"])

Score: 3  
Feedback: The candidate's answer does not address the question about handling missing data techniques.
Instead, they listed libraries without explaining how they relate to the topic.
To improve, the candidate should focus on specific techniques such as imputation methods, removal of missing values, or using algorithms that support missing data.
Model Answer: Common techniques for handling missing data include:  
1.
**Removing Rows/Columns**: If the missing data is minimal, you can remove the affected rows or columns.
2.
**Mean/Median/Mode Imputation**: Filling in missing values with the mean, median, or mode of the column.
3.
**Predictive Imputation**: Using machine learning algorithms to predict and fill in missing values based on other data.
4.
**K-Nearest Neighbors (KNN)**: Using the KNN algorithm to impute missing values based on the nearest neighbors.
5.
**Multiple Imputation**: Creating multiple datasets with different imputed values and averaging the results.
6.
**Fla

In [19]:
question = "Generate new question about deep learning"
generated_question = gen_llm().invoke({"input":question,"chat_history": chat_history, "context": ""})
chat_history.extend([HumanMessage(content=question), generated_question["answer"]])
question = generated_question['answer']

In [20]:
question

'What are the main challenges associated with training deep learning models, and how can they be addressed?'

In [21]:
chat_history

[HumanMessage(content='Generate new question about data science', additional_kwargs={}, response_metadata={}),
 'What are some common techniques for handling missing data in a dataset?',
 HumanMessage(content='Ptorch, scikit learn, numpy and pandas Library.', additional_kwargs={}, response_metadata={}),
 "Score: 3  \nFeedback: The candidate's answer does not address the question about handling missing data techniques. Instead, they listed libraries without explaining how they relate to the topic. To improve, the candidate should focus on specific techniques such as imputation methods, removal of missing values, or using algorithms that support missing data.  \nModel Answer: Common techniques for handling missing data include:  \n1. **Removing Rows/Columns**: If the missing data is minimal, you can remove the affected rows or columns.  \n2. **Mean/Median/Mode Imputation**: Filling in missing values with the mean, median, or mode of the column.  \n3. **Predictive Imputation**: Using mach

In [22]:
question = "do you know about LingoNet?"
generated_question = comunicate_llm().invoke({"input":question,"chat_history": chat_history, "context": ""})
chat_history.extend([HumanMessage(content=question), generated_question["answer"]])
response = generated_question['answer']
print_answer(response)

Yes, LingoNet is a lightweight transformer‑based language model designed specifically for on‑device text autocompletion on mobile.
During training, LingoNet was fine‑tuned on a multilingual corpus spanning English, Spanish, and Mandarin to support cross‑language suggestions.


In [23]:
question = "can I test LingoNet on a France dataset?"
generated_question = comunicate_llm().invoke({"input":question,"chat_history": chat_history, "context": ""})
chat_history.extend([HumanMessage(content=question), generated_question["answer"]])
response = generated_question['answer']
print_answer(response)

I don't know.


In [24]:
question = "do you know anything about LingoPer?"
generated_question = comunicate_llm().invoke({"input":question,"chat_history": chat_history, "context": ""})
chat_history.extend([HumanMessage(content=question), generated_question["answer"]])
response = generated_question['answer']
print_answer(response)

Yes, LingoPer is similar to LingoNet but additionally fine-tuned on Persian language data.


In [25]:
question = "Explain more LingoPer"
generated_question = comunicate_llm().invoke({"input":question,"chat_history": chat_history, "context": ""})
chat_history.extend([HumanMessage(content=question), generated_question["answer"]])
response = generated_question['answer']
print_answer(response)

LingoPer is a language model that builds upon the architecture of LingoNet, which is designed for text autocompletion.
The key distinction is that LingoPer has been specifically fine-tuned on data in the Persian language.
This fine-tuning allows it to better understand and generate text in Persian, making it more effective for applications that require Persian language processing.
It leverages the transformer architecture to provide accurate and contextually relevant suggestions for text input, similar to how LingoNet operates for other languages.
